In [ ]:
#0.jpg 파일 불러오기
from google.colab import files 

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'
        .format(name=fn, length=len(uploaded[fn]))) 

Saving music7.jpg to music7.jpg
User uploaded file "music7.jpg" with length 205688 bytes


In [ ]:
#1.라이브러리
import cv2 as cv
import numpy as np 
import copy
import matplotlib.pyplot as plt
import os
from google.colab.patches import cv2_imshow

In [ ]:
#1.이미지 이진화
#원본데이터 src 
src = cv.imread("./music6.jpg")
#악보 변환 : 그레이스케일 
grey_img = cv.cvtColor(src,cv.COLOR_BGR2GRAY)
#악보 변환 : 이진화
dumy , binary_img = cv.threshold(grey_img,180,255,cv.THRESH_BINARY_INV)



In [ ]:
#3.보표(stave)만 남기기
# 보표 검출 과 보표 정보 담기
mask = np.zeros(binary_img.shape,np.uint8)
cnt, dumy, stats, dumy = cv.connectedComponentsWithStats(binary_img)

stave_info=[]
for i in range(1, cnt):
  x, y, w, h, area = stats[i]
  if w > binary_img.shape[1] * 0.8:
    stave_info.append((x,y,w,h))
    cv.rectangle(mask, (x, y, w, h), (255, 0, 0), -1)

stave_img = cv.bitwise_and(binary_img, mask)

In [ ]:
#4.보표 한 줄(staff) 씩 나누기

#한 줄 위치 저장 (x,y,h)
seros=[]
for stave in stave_info:
  x,y,w,h=stave
  for j in range(x,x+w):
    pixels = 0
    for i in range(y,y+h):
      pixels+=(stave_img[i][j]==255)
    if pixels > h*0.5 :
      #mergeV1에서 추가
      #객체 세로 크기 추가, pixel로 순회하면 다 못 지움
      seros.append((j,y,pixels,h))



#세로 줄 정보 저장 (겹줄 파악) (x,y,w,h)
#mergeV1에서 추가: (x,y,w,pixel,staff_h)
#exception error (10/23): 마지막이 연속 줄일 때 안들어감
seros_info=[]
cnt=1

for i in range(len(seros)-1):
  if seros[i+1][0]==seros[i][0]+1:
    cnt+=1
  else:
    seros_info.append((seros[i][0]-cnt+1,seros[i][1],cnt,seros[i][2],seros[i][3]))
    cnt=1
#SOLVE : exception error (10/23): 마지막이 연속 줄일 때 안들어감 
if seros[-1][0]!=seros[-2][0]+1:
  seros_info.append((seros[-1][0],seros[-1][1],1,seros[-1][2],seros[i][3]))
else:
  seros_info.append((seros[-1][0]-cnt+1,seros[i][1],cnt,seros[i][2],seros[i][3]))


#세로 줄 삭제 및 오선 끊김 방지
staff_img=stave_img.copy()
for sero in seros_info:
  x,y,w,p,h = sero
  for j in range(x,x+w):
    for i in range(y,y+h):
     if stave_img[i][j+2*w]==255 and stave_img[i][j-2*w]==255:
        continue
     else:
        staff_img[i][j]=0





In [ ]:
#mergeV4에서 추가
#mask 연산 한 번 더함
#5.보표(stave)만 남기기

# 보표 검출 과 보표 정보 담기
mask = np.zeros(staff_img.shape,np.uint8)
cnt, dumy, stats, dumy = cv.connectedComponentsWithStats(staff_img)

for i in range(1, cnt):
  x, y, w, h, area = stats[i]
  if w > staff_img.shape[1] * 0.8:
    cv.rectangle(mask, (x, y, w, h), (255, 0, 0), -1)

staff_img = cv.bitwise_and(staff_img, mask)


In [ ]:
#mergeV1에서 추가
#보표별 이미지 저장 시 padding 주기
#삭제한 세로 선 길이 필요
sum_row=0
sum_col=0
for sero in seros_info:
  sum_row+=sero[3]
  sum_col+=sero[2]

sero_row = sum_row/len(seros_info)
sero_col = sum_col/len(seros_info)

padding_row = int(sero_row/8)
padding_col = int(sero_col)+1



In [ ]:
#5.오선 별로 따로 이미지 리스트에 저장

cnt, dumy, stats, centroids = cv.connectedComponentsWithStats(staff_img)  # 모든 객체 검출하기

staff_unit=[]
for i in range(1,cnt):
  (x,y,w,h,area) = stats[i]
  #mergeV2에서 추가: h는 padding_row 보다 커야 함
  if w < staff_img.shape[1]*0.8 or h<padding_row:
    continue
  else:
    img=np.zeros((w,h),np.uint8)
    #mergeV1에서 추가: padding 더함
    img=staff_img[y-padding_row:y+h+padding_row,x-padding_col:x+w+padding_col]
    staff_unit.append(img)


In [ ]:
#6.오선 위치 파악
# 각 보표의 한 줄 정보 저장lines=[ [staff1의 오선 정보],[ ],[ ],[ ] ]
lines=[]
for staff in staff_unit:
  height , width = staff.shape
  tmp=[]
  for row in range(height):
    pixels=0
    for col in range(width):
      pixels +=(staff[row][col]==255)
      if pixels>width*0.5:
        tmp.append(row)
        break
  lines.append(tmp)

#오선 정보 저장 (겹줄 파악)
#exception error (10/23): 마지막이 연속 줄일 때 안들어감
lines_info=[]
for line in lines:
  tmp=[]
  cnt=1
  for i in range(len(line)-1):
    if line[i+1]==line[i]+1:
      cnt+=1
    else:
      tmp.append((line[i]-cnt+1,cnt))
      cnt=1
  if line[-1]!=line[-2]+1:
    tmp.append((line[-1],1))
  else:
    tmp.append((line[-1]-cnt+1,cnt))
  lines_info.append(tmp)
  
#SOLVE : exception error (10/23): 마지막이 연속 줄일 때 안들어감 



#오선 간격 : standard에 사용
# mergeV1에서 추가 : 실행 셀 순서 변경...
#10.오선 간격으로 기준 정하기/ standarization
#나중에 상수로 쓴 거 다 stadarization 해야함
standard=0
sum=0
tmp=0
info = lines_info[0]
for i in range(len(info)-1):
  tmp = (info[i+1][0])-(info[i][0]+info[i][1]-1)
  sum+=tmp

standard= sum/(len(info)-1)



In [ ]:
#7.오선 지운 이미지 리스트

prepro_staff=[]
for k in range(len(staff_unit)):
  tmp_img=staff_unit[k].copy()
  tmp_info=lines_info[k]
  for line in tmp_info:
    y,h=line
    for i in range(y,y+h):
      for j in range(tmp_img.shape[1]):
        if (tmp_img[i+h][j]==255 and tmp_img[i-h][j]==255):
          continue
        else:
          tmp_img[i][j]=0
  prepro_staff.append(tmp_img)




In [ ]:
#8.close 연산으로 끊긴 부분 이어주기
close_staff=[]
mask = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
for staff in prepro_staff:
  tmp_staff = cv.morphologyEx(staff, cv.MORPH_CLOSE, mask)
  close_staff.append(tmp_staff)

In [ ]:
# 9.객체에 사각형 그려서 확인용
# 얕은 복사라 close_staff에도 사각형 그려져 있음
rect_staff=copy.deepcopy(close_staff)

for staff in rect_staff:
  cnt, dumy, stats, centroids = cv.connectedComponentsWithStats(staff)  # 모든 객체 검출하기
  for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]
    cv.rectangle(staff, (x, y, w, h), (255, 0, 0), 1)


In [ ]:
#11.보표 별로 객체 단위로 리스트에 담기
#image랑 stats 정보
#일단 객체 이미지랑 시작 열 위치랑 넓이 다 담기
#객체로 나누면 이제 다시 행렬 기준
every_obj=[]
for staff in close_staff:
  cnt, dumy, stats, centroids = cv.connectedComponentsWithStats(staff)  # 모든 객체 검출하기
  tmp=[]
  for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]
    img = staff[y-1:y+h+1,x-1:x+w+1]
    tmp.append([img,stats[i]])
  tmp.sort(key = lambda x:x[1][0])
  every_obj.append(tmp)

In [ ]:
#mergerV4에서 12,13순서 변경
#12. 점보다 작은 노이즈 제거
#mergeV1에서 추가
re1noise_obj=[]
for k in range(len(every_obj)):
  staff = every_obj[k]
  tmp=[]
  for obj in staff:
    img = obj[0]
    stats=obj[1]
    #점 보다 작을 때
    if stats[4]<(standard*0.5-1)*(standard*0.5-1):
      continue
    #숫자 or 점보다 크지만 area가 작아서 노이즈
    if (standard*0.5)*(standard*0.5)*0.8<=stats[4]<100:
         continue
    tmp.append([img,stats])
  re1noise_obj.append(tmp)
   


In [ ]:
#13.겹객체 노이즈
#위치로 겹 객체랑 숫자 없애기
#중심 점이 앞 혹은 뒤에 있는 객체 내부에 있는지(겹객체)
re2noise_obj=[]
for staff in re1noise_obj:
  tmp=[]
  tmp.append([staff[0][0],staff[0][1]])
  ind=1
  for i in range(1,len(staff)):
    img = staff[i][0]
    (x,y,w,h,area)=staff[i][1]
    Ci=y+h/2
    Cj=x+w/2
    #내부에 있음 (음자리표, 박자표)
    if (staff[i-ind][1][1]<Ci<staff[i-ind][1][1]+staff[i-ind][1][3])and (staff[i-ind][1][0]<Cj<staff[i-ind][1][0]+staff[i-ind][1][2]):
      ind+=1
      continue
    else:
      ind=1
    #위에 있음 (스타카토나 도돌이표)
    if (abs(x-staff[i-1][1][0])<standard)and (staff[i-1][1][0]<Cj<staff[i-1][1][0]+staff[i-1][1][2]):
      if staff[i-1][1][4] < 30:
        tmp.pop()
      continue
    tmp.append([img,staff[i][1]])
    
  re2noise_obj.append(tmp)


In [ ]:
#14.원본 보호
pitch_obj = copy.deepcopy(re2noise_obj)

In [ ]:
#15.함수
#직선 검사 함수 및 직선 제거
def count_line(img):

  seros=[]
  for j in range(img.shape[1]):
    pixels = 0
    for i in range(img.shape[0]):
      pixels+=(img[i][j]==255)
    if pixels >= img.shape[0]*0.5 :
      seros.append(j)

  if len(seros) <=1:
    if len(seros)==1:
      j=seros[0]
      for i in range(img.shape[0]):
        img[i][j]=0
    return len(seros)
  
  seros_info=[]
  cnt=1
  for i in range(len(seros)-1):
    if seros[i+1]==seros[i]+1:
      cnt+=1
    else:
      seros_info.append((seros[i]-cnt+1,cnt))
      cnt=1
  if seros[-1]!=seros[-2]+1:
    seros_info.append((seros[-1],1))
  else:
    seros_info.append((seros[-1]-cnt+1,cnt))

  for sero in seros_info:
    col,cnt = sero
    for w in range(cnt):
      for i in range(img.shape[0]):
        if img[i][col-1]==255:
          continue
        else:
          img[i][col+w]=0

  return len(seros_info)

#원 탐색 하는 함수 > [(행1,행2),(행1,행2)] >통과
def find_head(img):
  cnt, dumy, stats, centroids = cv.connectedComponentsWithStats(img)  # 모든 객체 검출하기
  stats=stats.tolist()
  stats.sort(key = lambda x:x[0])

  tmp=[]   
  for i in range(1,cnt):
    x,y,w,h,area =stats[i]
    #연속 꼬리 
    if h*2.5<=w and cnt>2:
       continue
    #노이즈
    if area<15:
        continue
    #하나 꼬리
    if area<80 and img.shape[0]>=standard*2.5:
        continue
    else:
       tmp.append((y,y+h))
  
  return tmp


In [ ]:
#16.음정을 숫자로 변환
#머리 범위 절대 위치로 변경
pitch_loc=[]
for k in range(len(pitch_obj)):
  note = pitch_obj[k]
  tmp=[]
  for m in range(len(note)):
    img=note[m][0]
    stat=note[m][1]
    #음자리표, 박자표
    if (k<=1 and m<=1) or (k>1 and m==0) :
        continue
    #음표 구분 (인덱스 sync 때문에 쉼표 필터링 안 함)
    #줄기 제거
    if standard*3.75<img.shape[0]:
       count_line(img)
    
    head = find_head(img)
    for i in range(len(head)):
      y1=head[i][0]+stat[1]
      y2=head[i][1]+stat[1]
      tmp.append((y1,y2))
  pitch_loc.append(tmp)

#오선 정보 추가 > 통과
add_line=[]
for line in lines_info:
  tmp=[]
  tmp.append(line[0][0]-standard)
  tmp.append(line[0][0]-standard*0.5)
  for i in range(len(line)):
    tmp.append(line[i][0])
    y=line[i][0]+line[i][1]-1
    tmp.append(y+standard*0.5)
  tmp.append(line[-1][0]+line[-1][1]-1+standard)
  add_line.append(tmp)

total_pitch=[]
for k in range(len(pitch_loc)):
  loc = pitch_loc[k]
  line= add_line[k]
  tmp=[]
  for i in range(len(loc)):
    y1,y2 = loc[i]
    for j in range(len(line)):
      if y1<line[j]<y2:
        tmp.append(13-j)
        break
  total_pitch.append(tmp)
   

In [ ]:
#17.원본 보호
beat_obj = copy.deepcopy(re2noise_obj)

In [ ]:
#18.함수
#직선 검사 함수 및 직선 제거
def count_line(img):

  seros=[]
  for j in range(img.shape[1]):
    pixels = 0
    for i in range(img.shape[0]):
      pixels+=(img[i][j]==255)
    if pixels >= img.shape[0]*0.5 :
      seros.append(j)

  if len(seros) <=1:
    if len(seros)==1:
      j=seros[0]
      for i in range(img.shape[0]):
        img[i][j]=0
    return len(seros)
  
  seros_info=[]
  cnt=1
  for i in range(len(seros)-1):
    if seros[i+1]==seros[i]+1:
      cnt+=1
    else:
      seros_info.append((seros[i]-cnt+1,cnt))
      cnt=1
  if seros[-1]!=seros[-2]+1:
    seros_info.append((seros[-1],1))
  else:
    seros_info.append((seros[-1]-cnt+1,cnt))

  for sero in seros_info:
    col,cnt = sero
    for w in range(cnt):
      for i in range(img.shape[0]):
        if img[i][col-1]== 255:
          continue
        img[i][col+w]=0

  return len(seros_info)

#객체 세는 함수 
def count_obj(img):
  cnt, dumy, stats, dumy = cv.connectedComponentsWithStats(img)
  count=0
  sum=0
  for i in range(1,cnt):
    x,y,w,h,area = stats[i]
    if area<15:
      continue
    count+=1
    sum+=area
  if sum <100:
    return 1
  else :
    return count 

#머리 색깔 검사 함수 : 채워지면 True , 비어있으면 False
def head_color(img):
  cnt, dumy, stats, dumy = cv.connectedComponentsWithStats(img)
  ind=1
  if cnt>2:
     m=max(stats[1][4],stats[2][4])
     if m==stats[2][4]:
        ind=2
  for j in range(stats[ind][2]):
    flag=0
    start=0
    for i in range(stats[ind][3]):
      if img[stats[ind][1]+i][stats[ind][0]+j]==255 and start==0 and flag==0:
        start=1
      elif start==1 and img[stats[ind][1]+i][stats[ind][0]+j]==0 and flag==0:
        flag=1
      elif start==1 and flag==1 and img[stats[ind][1]+i][stats[ind][0]+j]==255:
          return False
  return True
  

In [ ]:
#16. 박자 분류 : 음표를 숫자로 변환
# 점[0]
# 온음표[4], 2분음표[2], 4분음표[1], 8분음표[0.5]
# 4분쉼표[-4], 2분쉼표[-2], 4분쉼표[-1], 8분쉼표[-0.5]

total_beat=[]
for k in range(len(beat_obj)):
  note = beat_obj[k]
  line = lines_info[k]
  tmp=[]
  for m in range(len(note)):
    img=note[m][0]
    (x,y,col,row,area)=note[m][1]
    #row , col = obj.shape

    if (k<=1 and m<=1) or (k>1 and m==0) :
        continue
    if area < (standard*0.5+1)*(standard*0.5+1):
        tmp.append(0)

    elif standard*3.75<row :
        line_cnt=count_line(img)
        if line_cnt>=2:
          for i in range(line_cnt):
             tmp.append(0.5)
       
        else:
           obj_cnt=count_obj(img)
           if obj_cnt>=2:
             tmp.append(0.5)
           
           else:
             clr=head_color(img)
             if clr == True:
               tmp.append(1)
             
             else:
               tmp.append(2)
    else:
      if standard*1.5<col:
        tmp.append(4)

      elif row<standard:
        line_loc = line[1][0]+line[1][1]-1
        y_loc = y
        if y_loc-line_loc>2:
          tmp.append(-2)
        else:
          tmp.append(-4)
          
      elif row<standard*2.25:
        tmp.append(-0.5)

      else:
        tmp.append(-1)

  total_beat.append(tmp)

    

In [ ]:
#박자 리스트 업데이트 : 점 박자 처리
#음정 리스트 업데이트 : 점 인덱스 음정 삭제, 쉼표 음정 0
beatV1=[]
pitchV1=[]
for i in range(len(total_beat)):
  tmp_b=[]
  tmp_p=[]
  for j in range(len(total_beat[i])):
    if total_beat[i][j]==0:
      tmp_b[-1]=tmp_b[-1]*1.5
    else:
      if total_beat[i][j] <0:
        tmp_b.append(total_beat[i][j]*(-1))
        tmp_p.append(0)
      else:
        tmp_b.append(total_beat[i][j])
        tmp_p.append(total_pitch[i][j])
  beatV1.append(tmp_b)
  pitchV1.append(tmp_p)

In [ ]:
pitch_R=[]
pitch_L=[]
Bias_R=35
Bias_L=3
for i in range(len(pitchV1)):
  if i%2==0:
    if i==0:
      pitch_R.append(991)
    for j in range(len(pitchV1[i])):
        tmp = pitchV1[i][j]
        mod = tmp%7
        div = tmp//7
        if tmp==0:
          pitch_R.append(65)
        elif mod ==1:
          pitch_R.append(1+Bias_R+12*div)
        elif mod ==2:
          pitch_R.append(3+Bias_R+12*div)
        elif mod ==3:
          pitch_R.append(5+Bias_R+12*div)
        elif mod ==4:
          pitch_R.append(6+Bias_R+12*div)
        elif mod ==5:
          pitch_R.append(8+Bias_R+12*div)
        elif mod ==6:
          pitch_R.append(10+Bias_R+12*div)
        elif mod ==0 and tmp!=0:
          pitch_R.append(12+Bias_R+12*(div-1))
    if i==len(pitchV1)-2:
      pitch_R.append(991)

  else:
    if i==1:
      pitch_L.append(992)
    for j in range(len(pitchV1[i])):
        tmp = pitchV1[i][j]
        mod= tmp%7
        div = tmp//7
        if tmp==0:
          pitch_L.append(65)
        elif mod ==1:
          pitch_L.append(1+Bias_L+12*div)
        elif mod ==2:
          pitch_L.append(2+Bias_L+12*div)
        elif mod ==3:
          pitch_L.append(4+Bias_L+12*div)
        elif mod ==4:
          pitch_L.append(6+Bias_L+12*div)
        elif mod ==5:
          pitch_L.append(8+Bias_L+12*div)
        elif mod ==6:
          pitch_L.append(9+Bias_L+12*div)
        elif mod ==0 and tmp!=0:
          pitch_L.append(11+Bias_L+12*(div-1))
    if i==len(pitchV1)-1:
      pitch_L.append(992)

    



In [ ]:
beat_R=[]
beat_L=[]
K=2
for i in range(len(beatV1)):
  if i%2==0:
    if i==0:
      beat_R.append(993)
    for j in range(len(beatV1[i])):
        tmp = beatV1[i][j]*K
        beat_R.append(int(tmp))
    if i==len(beatV1)-2:
        beat_R.append(993)

  else:
    if i==1:
      beat_L.append(994)
    for j in range(len(beatV1[i])):
        tmp = beatV1[i][j]*K
        beat_L.append(int(tmp))
    if i==len(beatV1)-1:
        beat_L.append(994)


In [ ]:
print(pitch_R)
print(pitch_L)

[991, 43, 43, 41, 40, 41, 43, 47, 48, 45, 47, 48, 43, 47, 48, 41, 41, 40, 38, 40, 41, 43, 36, 38, 40, 41, 40, 38, 36, 38, 40, 41, 43, 47, 47, 45, 43, 40, 40, 38, 36, 38, 991]
[992, 12, 14, 16, 12, 17, 16, 14, 19, 17, 16, 21, 14, 16, 17, 19, 17, 16, 21, 14, 19, 992]


In [ ]:
print(beat_R)
print(beat_L)

[993, 4, 1, 1, 1, 1, 6, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 4, 993]
[994, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 1, 1, 2, 2, 2, 4, 4, 4, 4, 994]
